In [1]:
import pandas as pd
import numpy as np

In [2]:
# func 모음

# 1. 종목코드를 6자리로 통일. ex) 990 -> 000990
def padding(x):
    x = str(x)
    return x.rjust(6, "0")

# 2. 결산월 12월인 것만 선택
# : 전체 데이터 4963개 중에서 12월 결산인 데이터가 4836개이다. 나머지 결산월의 데이터가 전체의 2.5%로 데이터량이 작기도하고 결측치가 많아 제거
def func_yearin(x):
    if x[5:7] == '12':
        return True
    else:
        return False

# 3. 회계년도컬럼, 년도만 뽑아서 int화
def change_year(x):
    return int(x[:4])

# 4. 연결, 개별재무제표 통합을 위해 연결재무 데이터 컬럼명 통일화 : "(IFRS연결)"제거
def func_replace_col1(x):
    x = x.replace(" ","")
    x = x.replace(",","")
    x = x.replace("(비교)","")
    x = x.replace("(IFRS)","")
    x = x.replace("(*)","")
    x = x.replace("[제조]","")
    return x.replace("(IFRS연결)","")

# 5. 연결, 개별재무제표 통합을 위해 개별재무 데이터 컬럼명 통일화 : "(IFRS)"제거
def func_replace_col2(x):
    x = x.replace(" ","")
    x = x.replace(",","")
    x = x.replace("(비교)","")
    x = x.replace("(IFRS)","")
    x = x.replace("(*)","")
    x = x.replace("[제조]","")
    return x.replace("(IFRS)","")

# 6. 2020년 raw data 제외
def func_rm_2020(x):
    if x[:4] == "2020":
        return False
    else:
        return True

# 7. True, False 반전
def func_not(x):
    return not x

In [8]:
개별W = pd.read_csv('./datasets/raw/개별ww.csv',encoding='cp949')
연결W = pd.read_csv('./datasets/raw/연결ww.csv',encoding='cp949')
개별_산업성장률= pd.read_csv("개별산업매출성장률.csv", encoding='UTF8',index_col=0)
연결_산업성장률= pd.read_csv("연결산업매출성장률.csv", encoding='UTF8',index_col=0)

개별W["거래소코드"] = 개별W["거래소코드"].map(padding) # 거래소코드 padding
개별W["회계년도"] = 개별W["회계년도"].map(change_year) # 년도만 표시

연결W["거래소코드"] = 연결W["거래소코드"].map(padding) # 거래소코드 padding
연결W["회계년도"] = 연결W["회계년도"].map(change_year) # 년도만 표시

#WW 지수 = -0.091(현금흐름 / 총자산) - 0.062*(배당금을 지급하면 1을 가지는 더미변수) +0.021 * (장기차입금/총자산) - 0.044 * (총자산에 자연로그를 취한값) + 0.102 * (기업이 속한 산업의 매출성장률)+0.035 * (기업의 매출성장률)

In [9]:
개별WW = pd.merge(개별W, 개별_산업성장률, on=["회계년도","상장협 산업분류 코드(대분류)"],how = 'left')
연결WW = pd.merge(연결W, 개별_산업성장률, on=["회계년도","상장협 산업분류 코드(대분류)"],how = 'left')

In [10]:
연결WW.columns = 연결WW.columns.map(func_replace_col1)
개별WW.columns = 개별WW.columns.map(func_replace_col2)

In [11]:
# ww지수 -> 자산로그화
개별WW['총자산자연로그']=''
연결WW['총자산자연로그']=''

In [13]:
from numpy import log as ln
for i in range(len(개별WW)):
  개별WW['총자산자연로그'][i] = ln(개별WW['자산'][i])

def func_float(x):
  return float(x)
개별WW['총자산자연로그'] = 개별WW['총자산자연로그'].map(func_float)

C:\Users\신민경\AppData\Local\Temp\ipykernel_52588\1706874245.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  개별WW['총자산자연로그'][i] = ln(개별WW['자산'][i])


In [14]:
from numpy import log as ln
for i in range(len(연결WW)):
  연결WW['총자산자연로그'][i] = ln(연결WW['자산'][i])

def func_float(x):
  return float(x)
연결WW['총자산자연로그'] = 연결WW['총자산자연로그'].map(func_float)

C:\Users\신민경\AppData\Local\Temp\ipykernel_52588\2787238940.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  연결WW['총자산자연로그'][i] = ln(연결WW['자산'][i])


In [15]:
## 중복된 값 제거
연결WW.drop_duplicates(['거래소코드',"회계년도"], keep='last', inplace=True)
연결WW.reset_index(drop=True, inplace=True)

## 외국계기업제거
def func_isnt_fcompany(x):
    if x[:1] == '9':
        return False
    else:
        return True

연결WW = 연결WW[연결WW["거래소코드"].map(func_isnt_fcompany)]

# 중복 거래소코드 제거
def func_overlap_code(x):
    if x == (연결WW[연결WW["회사명"]==company_name]["거래소코드"].iloc[-1]) :
        return True
    else:
        return False

list_companyname = list(연결WW["회사명"].unique())
dfs = list()


for i in list_companyname:
    company_name = i
    #각 기업별로 데이터프레임 생성
    df_concat = 연결WW[연결WW["회사명"]==i]
    df_concat = df_concat[df_concat["거래소코드"].map(func_overlap_code)] # 거래소코드 바뀌기 전의 거래소코드의 데이터는 삭제
    dfs.append(df_concat)
연결WW = pd.concat(dfs)
연결WW.reset_index(drop=True, inplace=True)


## 기업명 변경
연결WW[연결WW['거래소코드']=='036420']
연결WW.loc[연결WW['회사명']=='(주)제이콘텐트리',['회사명']] = '(주)콘텐트리중앙'

In [16]:
## 중복된 값 제거
개별WW.drop_duplicates(['거래소코드',"회계년도"], keep='last', inplace=True)
개별WW.reset_index(drop=True, inplace=True)

## 외국계기업제거
def func_isnt_fcompany(x):
    if x[:1] == '9':
        return False
    else:
        return True

개별WW = 개별WW[개별WW["거래소코드"].map(func_isnt_fcompany)]

# 중복 거래소코드 제거
def func_overlap_code(x):
    if x == (개별WW[개별WW["회사명"]==company_name]["거래소코드"].iloc[-1]) :
        return True
    else:
        return False

list_companyname = list(개별WW["회사명"].unique())
dfs = list()


for i in list_companyname:
    company_name = i
    #각 기업별로 데이터프레임 생성
    df_concat = 개별WW[개별WW["회사명"]==i]
    df_concat = df_concat[df_concat["거래소코드"].map(func_overlap_code)] # 거래소코드 바뀌기 전의 거래소코드의 데이터는 삭제
    dfs.append(df_concat)
개별WW = pd.concat(dfs)
개별WW.reset_index(drop=True, inplace=True)


## 기업명 변경
개별WW[개별WW['거래소코드']=='036420']
개별WW.loc[개별WW['회사명']=='(주)제이콘텐트리',['회사명']] = '(주)콘텐트리중앙'

In [19]:
## 배당금 NAN -> 0
개별WW['배당금_현금']=개별WW['배당금_현금'].fillna(0)
연결WW['배당금_현금']=연결WW['배당금_현금'].fillna(0)

In [20]:
연결WW.isna().sum()

회사명                     0
거래소코드                   0
회계년도                    0
상장협산업분류코드(대분류)          0
상장협산업분류(대분류)            0
자산                   4754
장기차입금                4754
영업활동으로인한현금흐름(간접법)    4754
매출액증가율               4800
배당금_현금                  0
산업매출성장률                 0
총자산자연로그              4754
dtype: int64

In [21]:
개별WW.isna().sum()

회사명                    0
거래소코드                  0
회계년도                   0
상장협산업분류코드(대분류)         0
상장협산업분류(대분류)           0
자산                   378
장기차입금                378
영업활동으로인한현금흐름(간접법)    378
매출액증가율               450
배당금_현금                 0
산업매출성장률                0
총자산자연로그              378
dtype: int64

In [22]:
# df_재무를 완전데이터와 결측데이터로 나눈 뒤 결측데이터를 개별재무데이터로 대체시키고, df_재무로 다시 통합하면 끝!
WW완전 =연결WW[연결WW["매출액증가율"].isna().map(func_not)]
# df_재무_완전

WW결측 =연결WW[연결WW["매출액증가율"].isna()]
# df_재무_결측

list_na_col = ['상장협산업분류코드(대분류)','상장협산업분류(대분류)','자산','장기차입금','영업활동으로인한현금흐름(간접법)','매출액증가율','배당금_현금','산업매출성장률','총자산자연로그']

WW결측 = WW결측.drop(list_na_col,axis=1)
WW결측 = pd.merge(WW결측,개별WW[['거래소코드','회계년도']+list_na_col],on=['거래소코드','회계년도'],how='left')

WW_df  = pd.concat([WW완전,WW결측],axis=0)

In [23]:
WW_df.isna().sum()

회사명                    0
거래소코드                  0
회계년도                   0
상장협산업분류코드(대분류)         2
상장협산업분류(대분류)           2
자산                   377
장기차입금                377
영업활동으로인한현금흐름(간접법)    377
매출액증가율               448
배당금_현금                 2
산업매출성장률                2
총자산자연로그              377
dtype: int64

In [24]:
print( '결측치가 존재하는 기업의 수',len(WW_df[WW_df.isna().any(1)].회사명.unique()),'개')

결측치가 존재하는 기업의 수 248 개


C:\Users\신민경\AppData\Local\Temp\ipykernel_52588\4262142544.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  print( '결측치가 존재하는 기업의 수',len(WW_df[WW_df.isna().any(1)].회사명.unique()),'개')


In [25]:
## 결측치 제거
WW_df = WW_df[WW_df["매출액증가율"].isna().map(func_not)] # 위 결측치 없는 데이터로 필터링
WW_df

WW_df.isna().sum().to_dict()

{'회사명': 0,
 '거래소코드': 0,
 '회계년도': 0,
 '상장협산업분류코드(대분류)': 0,
 '상장협산업분류(대분류)': 0,
 '자산': 0,
 '장기차입금': 0,
 '영업활동으로인한현금흐름(간접법)': 0,
 '매출액증가율': 0,
 '배당금_현금': 0,
 '산업매출성장률': 0,
 '총자산자연로그': 0}

In [26]:
WW_df.dropna(inplace=True)

In [27]:
## WW 지수 구하기
# 배당금 지급하면 1을 가지는 더미변수 -> WW지수를 위해

def func_배당금여부(x):
  if x==0:
    return 0
  else:
    return 1

In [29]:
WW_df['배당금지급_여부']=WW_df['배당금_현금'].map(func_배당금여부)

In [30]:
WW_df.drop(columns = ['배당금_현금'],inplace=True)

In [32]:
#WW 지수 = -0.091(현금흐름 / 총자산) - 0.062*(배당금을 지급하면 1을 가지는 더미변수) +0.021 * (장기차입금/총자산) - 0.044 * (총자산에 자연로그를 취한값) + 0.102 * (기업이 속한 산업의 매출성장률)+0.035 * (기업의 매출성장률)
# 현금흐름은 영업에서 창출된 현금흐름
WW_df['WW지수'] =-0.091 * (WW_df['영업활동으로인한현금흐름(간접법)']*1000000/WW_df['자산']*1000000) - 0.062 * WW_df['배당금지급_여부'] + 0.021*(WW_df['장기차입금']*1000000/WW_df['자산']*1000000) - 0.044 * WW_df['총자산자연로그'] + 0.102 * WW_df["산업매출성장률"] + 0.035 * WW_df['매출액증가율']

In [33]:
WW_df.drop(['상장협산업분류코드(대분류)','상장협산업분류(대분류)','자산','장기차입금','영업활동으로인한현금흐름(간접법)','매출액증가율','배당금지급_여부','산업매출성장률','총자산자연로그'],axis=1,inplace=True)

In [34]:
WW_df.reset_index(drop=True,inplace=True)

In [35]:
WW_df.to_csv('WW지수.csv',encoding='UTF-8-sig')